In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pycbc.frame import read_frame
from pycbc import psd as psd_func
from pycbc.noise import noise_from_psd
from pycbc.filter import sigma
from simple_pe.waveforms import make_waveform, calculate_mode_snr

import logging
_logger = logging.getLogger('PESummary')
_logger.setLevel(logging.CRITICAL + 10)
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

/home/ben.patterson/.conda/envs/igwn_eccentric/lib/python3.10/site-packages/pycbc/types/array.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal as _lal


lal.MSUN_SI != Msun
__name__ = gwsurrogate.new.spline_evaluation
__package__= gwsurrogate.new


In [2]:
# Directory to gwf files
zero_noise_gwfs = 'MA_datafind_injections/zero_noise/output/'

# Generate psd
ifos = ['H1', 'L1', 'V1']
tlen = 32
sample_rate = 8192
f_low = 20
psds = {'H1': 'aLIGO175MpcT1800545',
        'L1': 'aLIGO175MpcT1800545',
        'V1': 'AdVO3LowT1800545',
        'f_low': f_low,
        'f_high': int(sample_rate/2),
        'length': tlen,
        'delta_f': 1. / tlen
        }
psd = {}
for ifo in ifos:
    psd[ifo] = psd_func.analytical.from_string(psds[ifo], psds['length'] * psds['f_high'] + 1, psds['delta_f'],
                                                     psds['f_low'])

# Read in gwf files as pycbc timeseries
zero_noise_hs = {}
gaussian_noise_hs = {}
seed = 3000
seed_mod = {'H1': 0, 'L1': 1, 'V1': 2}
for ifo in ifos:

    # Read files for zero noise data
    zero_noise_hs[ifo] = read_frame(zero_noise_gwfs+f'{ifo}-INJECTION.gwf', f'{ifo}:HWINJ_INJECTED')
    time_shift = zero_noise_hs[ifo].sample_times[np.argmax(np.array(zero_noise_hs[ifo]))]
    zero_noise_hs[ifo].start_time -= time_shift
    zero_noise_hs[ifo] = zero_noise_hs[ifo].time_slice(-31, 1)

    # Add noise to get gaussian noise data
    gaussian_noise_hs[ifo] = zero_noise_hs[ifo].copy()
    noise = noise_from_psd(len(gaussian_noise_hs[ifo]), 1/sample_rate, psd[ifo], seed=seed+seed_mod[ifo])
    noise.start_time = gaussian_noise_hs[ifo].start_time
    gaussian_noise_hs[ifo] += noise

In [3]:
# Generate eccentric harmonics
harm_pars = {'ecc10sqrd': 0.2**2, 'mass_1': 40, 'mass_2': 20, 'spin_1x': 0, 'spin_1y': 0, 'spin1z': 0.2, 'spin_2x': 0, 'spin_2y': 0, 'spin2z': 0.1, 'chi_eff': 0, 'distance': 1000}
df = 1/tlen
flen = (tlen*sample_rate)//2 + 1
harms = make_waveform(harm_pars, df, f_low, flen, approximant='TEOBResumS-Dali-Harms', n_ecc_harms=4, n_ecc_gen=6, ecc_harm_psd=psd)

In [4]:
# Get SNR in each mode
zero_sigmas = {}
zero_rhos = {}
gaussian_rhos = {}
for ifo in ifos:
    zero_sigmas[ifo] = sigma(zero_noise_hs[ifo], psd=psd[ifo], low_frequency_cutoff=f_low)
    zero_rhos[ifo], _ = calculate_mode_snr(zero_noise_hs[ifo], psd[ifo], harms[ifo], zero_noise_hs[ifo].sample_times[0],
                                           zero_noise_hs[ifo].sample_times[-1], f_low, harms[ifo].keys(), dominant_mode=0, subsample_interpolation=False)
    gaussian_rhos[ifo], _ = calculate_mode_snr(gaussian_noise_hs[ifo], psd[ifo], harms[ifo], gaussian_noise_hs[ifo].sample_times[0],
                                               gaussian_noise_hs[ifo].sample_times[-1], f_low, harms[ifo].keys(), dominant_mode=0, subsample_interpolation=False)

# Print SNRs
print('Zero noise')
for ifo in ifos:
    print(f'{ifo}: {zero_sigmas[ifo]:.2f}')
    total_sq = 0
    for key in zero_rhos[ifo].keys():
        print(f'  rho_{key}: {np.abs(zero_rhos[ifo][key]):.2f}, phi_{key}: {np.angle(zero_rhos[ifo][key]):.2f}')
        total_sq += np.abs(zero_rhos[ifo][key])**2
    print(f'  total: {np.sqrt(total_sq):.2f}')
print('\nGaussian noise')
for ifo in ifos:
    print(f'{ifo}:')
    total_sq = 0
    for key in gaussian_rhos[ifo].keys():
        print(f'  rho_{key}: {np.abs(gaussian_rhos[ifo][key]):.2f}, phi_{key}: {np.angle(gaussian_rhos[ifo][key]):.2f}')
        total_sq += np.abs(gaussian_rhos[ifo][key])**2
    print(f'  total: {np.sqrt(total_sq):.2f}')

Zero noise
H1: 18.78
  rho_0: 11.82, phi_0: -1.46
  rho_1: 2.54, phi_1: 2.61
  rho_-1: 6.13, phi_-1: -0.65
  rho_2: 0.80, phi_2: 0.55
  total: 13.58
L1: 23.63
  rho_0: 14.87, phi_0: 1.97
  rho_1: 3.23, phi_1: -0.25
  rho_-1: 7.73, phi_-1: 2.79
  rho_2: 1.00, phi_2: -2.30
  total: 17.10
V1: 5.81
  rho_0: 4.91, phi_0: -2.06
  rho_1: 1.05, phi_1: 2.09
  rho_-1: 1.25, phi_-1: -0.88
  rho_2: 0.40, phi_2: -0.15
  total: 5.19

Gaussian noise
H1:
  rho_0: 11.65, phi_0: -1.56
  rho_1: 2.23, phi_1: 1.97
  rho_-1: 8.06, phi_-1: -0.58
  rho_2: 1.45, phi_2: 0.39
  total: 14.42
L1:
  rho_0: 15.17, phi_0: 2.06
  rho_1: 3.83, phi_1: -0.37
  rho_-1: 8.17, phi_-1: 2.82
  rho_2: 2.60, phi_2: 2.39
  total: 17.84
V1:
  rho_0: 5.41, phi_0: -2.03
  rho_1: 1.85, phi_1: 2.65
  rho_-1: 1.06, phi_-1: -0.06
  rho_2: 3.03, phi_2: -0.23
  total: 6.56


In [5]:
plt.plot(zero_noise_hs['H1'].sample_times, zero_noise_hs['H1'])
plt.plot(zero_noise_hs['L1'].sample_times, zero_noise_hs['L1'])
plt.plot(zero_noise_hs['V1'].sample_times, zero_noise_hs['V1'])
plt.xlim(-1, 0.1)
plt.show()

plt.plot(gaussian_noise_hs['H1'].sample_times, gaussian_noise_hs['H1'])
plt.plot(gaussian_noise_hs['L1'].sample_times, gaussian_noise_hs['L1'])
plt.xlim(gaussian_noise_hs['H1'].sample_times[0], gaussian_noise_hs['H1'].sample_times[0]+0.2)
plt.show()

FileNotFoundError: Matplotlib's TeX implementation searched for a file named 'phvr7t.tfm' in your texmf tree, but could not find it

<Figure size 640x480 with 1 Axes>

FileNotFoundError: Matplotlib's TeX implementation searched for a file named 'phvr7t.tfm' in your texmf tree, but could not find it

<Figure size 640x480 with 1 Axes>